In [1]:
from athina_evals.llms.openai_service import OpenAiService
from athina_evals.evaluators.llm.answer_relevance.evaluator import AnswerRelevanceEvaluator

In [2]:
model = "gpt-3.5-turbo"
user_query = "Who is the president of the United States?"
prompt = [
    {
        "role": "user",
        "content": user_query
    }
]

openai_service = OpenAiService()
response = openai_service.chat_completion(prompt, model=model)

In [3]:
AnswerRelevanceEvaluator(
    model="gpt-3.5-turbo"
).run(user_query=user_query, response=response)

{'failure': False,
 'reason': "The response answers the user's query sufficiently. It provides the name of the current president of the United States, which is Joe Biden."}